In [1]:
from config import fund_dict
import requests
import pandas as pd
from helper import create_holdings_df, process_scraped_data
import traceback
import numpy as np
import seaborn as sns
import traceback

figi_api_key = '34dada0c-5a22-4f5a-a095-739afe8fe591' 

In [2]:
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df()
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print(fund_name, cik)
    except:
        print(traceback.format_exc())
        continue
df_all['nameOfIssuer_link'] =df_all['nameOfIssuer'].apply(
    lambda x: str('''<a href="http://www.google.com/search?q=stock price {}">{}</a>''' \
        .format(x, x)))
    

df_heavy, df_hot, df_cold = process_scraped_data(df_all)

https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
AKRE CAPITAL MANAGEMENT 1112520
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ANSON FUNDS 1491072
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
APPALOOSA LP 1656456
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/153

/Users/willsolo/Documents/GitHub/stockFries_gha/notebooks/helper.py:97: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
/Users/willsolo/Documents/GitHub/stockFries_gha/notebooks/helper.py:98: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)


In [5]:
df_all.tail()
#set(df_all.sshPrnamtType.tolist())

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,nameOfIssuer_link
50003,WINVEST ACQUISITION CORP,*W EXP 08/09/202,97655B117,4843,"{'sshPrnamt': '974500', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '974500', 'Shared': '0', 'None': '0'}",974500,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50004,XPAC ACQUISITION CORP,CLASS A ORD,G9831X106,4880,"{'sshPrnamt': '500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '500000', 'Shared': '0', 'None': '0'}",500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50005,YOTTA ACQUISITION CORPORATIO,UNIT 03/15/2027,98741Y202,4487,"{'sshPrnamt': '450000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '450000', 'Shared': '0', 'None': '0'}",450000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50006,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,120,"{'sshPrnamt': '1500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '1500000', 'Shared': '0', 'None': '0'}",1500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50007,ZENDESK INC,COM,98936J101,11701,"{'sshPrnamt': '157974', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '157974', 'Shared': '0', 'None': '0'}",157974,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."


In [ ]:
df_recent_options = pd.read_json(hedge_fund_json_dg, orient='split')
df_puts_only = df_recent_options.drop_duplicates(subset=['CUSIP','HF_Name'], keep=False)
df_puts_only = df_puts_only[(df_puts_only['Option_Type']=='Put') & (df_puts_only['HF_Name']!='CAPITAL FUND MANAGEMENT')]
print(len(set(df_puts_only.HF_Name.tolist())))
df_puts_only.sort_values('Company').reset_index().tail(100)
df_puts_only.groupby(['CUSIP','Company','Option_Type'],as_index=False).agg(
        {'Shares': 'sum', 'Value': 'sum', 'HF_Name': 'unique'}).sort_values('Value').tail(20)